In [1]:
import pandas as pd
import numpy as np
import math
import yaml

### Help Functions

In [2]:
def calculate_amount(unit_price, budget):
    amount = budget / unit_price
    return amount

def calculate_profit(buy_price, sell_price):
    profit = sell_price - buy_price
    return profit

def get_weighted_list_length(list_item, weight):
    list_length = int(math.ceil(len(list_item) * weight))
    return list_length

def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)
    
def process(src):
    return pd.read_csv(src, delimiter=',',index_col=0)

## Weighted position investment function

In [3]:
def weighted_position_investing(input_dataframe, investment_capital, buy_weight, sell_weight):
    """
    Args:
        input_dataframe: pandas.DataFrame
                    Column position [0] needs to be close price.
                    All other columns need to consist of "Buy"/"Sell"/"Hold" decisions.
        investment_capital: Float
                    The amount of "money" able to be invested.
        buy_weight: Float
                    Values must be between 0-1. Represents the amount to invest from
                    available remaining capital (in procentage). Ie, 0.5 -> 50% of remaining
                    available capital on buy.
        sell_weight: Float
                    Values must be between 0-1. Represents the amount of invested capital
                    to sell out of (in procentage). Ie, 0.5 -> 50% of invested capital on sell.
    Returns:
        pandas.DataFrame
            Profit calculations appended to input_dataframe.
    """
    
    output_dataframe = pd.DataFrame(index=input_dataframe.index, columns=input_dataframe.columns[1:])
    output_dataframe = output_dataframe.rename(columns={"random_forest": "random_forest_profit","svc":"svc_profit",
                              "logreg":"logreg_profit", "gauss_nb": "gauss_nb_profit"})
    
    # LOOP THROUGH COLUMNS
    for col in input_dataframe.columns[1:]:
        current_profit = 0
        holding_value = 0
        resulting_col = col
        
        wallet = investment_capital
        positions_close = []
        positions_amount = []
        investment_amount = wallet * buy_weight
    
        # LOOP THROUGH EACH ROW
        for i in range(len(input_dataframe)):
            
            current_close_price = input_dataframe["close"].iloc[i]
            label = input_dataframe[col].iloc[i]
            
            # ON BUY
            if((wallet > 0) & (label == "buy")):
                # check to see if invest wallet amount or investment amount.
                if wallet < investment_amount:
                    amount = calculate_amount(current_close_price, wallet)
                    positions_close.append(current_close_price)
                    positions_amount.append(amount)
                    # remove money from wallet
                    wallet -= wallet
                    investment_amount = wallet * buy_weight
                    
                else:
                    amount = calculate_amount(current_close_price, investment_amount)
                    positions_close.append(current_close_price)
                    positions_amount.append(amount)
                    # remove money from wallet
                    wallet -= investment_amount
                    investment_amount = wallet * buy_weight

                # ADD NEW PROFIT TO DATAFRAME
                output_dataframe[resulting_col].iloc[i] = current_profit
                

            # ON SELL
            elif((len(positions_close) > 0) & (label == "sell")):
                n = get_weighted_list_length(positions_close, sell_weight)
                for close_p, amount_p in zip(positions_close[:n], positions_amount[:n]):
                    # get sell price and compare to buy price
                    selling_price = current_close_price * amount_p
                    buying_price = close_p * amount_p
                    # update wallet to have money again, i.e selling_price
                    wallet += selling_price
                    # register profit/loss
                    profit = calculate_profit(buying_price, selling_price)
                    current_profit += profit
                investment_amount = wallet * buy_weight
                    
                # ADD NEW PROFIT TO DATAFRAME
                output_dataframeres[resulting_col].iloc[i] = current_profit
                
                # EMPTY POSITIONS LISTS
                positions_close = positions_close[n:]
                positions_amount = positions_amount[n:]

            # ON HOLD
            else:
                # Since we are holding, do nothing except udpate current profit
                # ADD NEW PROFIT TO DATAFRAME
                output_dataframeres[resulting_col].iloc[i] = current_profit
    
        # SELL OFF OUT REMAINING POSITIONS
        current_close_price = input_dataframe["close"].iloc[-1]
        for close_p, amount_p in zip(positions_close, positions_amount):
            selling_price = current_close_price * amount_p
            buying_price = close_p * amount_p
            profit = calculate_profit(buying_price, selling_price)
            current_profit += profit
        output_dataframe[resulting_col].iloc[-1] = current_profit
    
    return output_dataframe

### Multiple profile calculations function:

In [4]:
def multiple_profiles_calculation(yaml_configs):
    configs = load_yaml(yaml_configs)
    profiles = configs['investing_profiles']
    data = process(configs['data'])
    investment_capital = configs['investment_capital']
    profiles
    results = []
    for profile in profiles:
        buy_weight = profiles[profile]['buy_weight']
        sell_weight = profiles[profile]['sell_weight']
        result = weighted_position_investing(data, investment_capital,buy_weight, sell_weight)
        results.append(result)
    return results
    
    

### Example use  of multi-position investment function:

In [5]:
temp_configs = load_yaml('profit_calculation_config.yaml')
temp_data = process(temp_configs['data'])
multi_example = weighted_position_investing(temp_data, 100,1,1) # 1 = 100% usage of money, 0.5 = 50% useage of money.
multi_example.tail(1)

KeyError: 'random_forest'

### Example use of profiles_calculation:

In [ ]:
all_profiles = multiple_profiles_calculation('profit_calculation_config.yaml')
len(all_profiles)

In [ ]:
all_profiles[0].tail(1)

In [ ]:
all_profiles[1].tail(1)

In [ ]:
all_profiles[2].tail(1)